In [100]:
## Importing Spark

import pyspark
import sys
import re
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from operator import add
from pprint import pprint
import pandas as pd
import numpy as np
sc = pyspark.SparkContext.getOrCreate()

# Setting up news-api
# Init
import json
import requests
import os
from path import Path

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
nltk.download('vader_lexicon')
nltk.download('stopwords')


from newsapi import NewsApiClient
from newsapi.newsapi_auth import NewsApiAuth
from newsapi import const
from newsapi.newsapi_exception import NewsAPIException
newsapi = NewsApiClient(api_key='19b1ffe70b4e4cf583d1089316fbf9c6')

#
#https://medium.com/daily-python/python-script-to-search-for-news-based-on-keywords-daily-python-5-509348bd190e


#Reference Github Code:
#https://github.com/Ava33343/nlp-homework/blob/ed6728b7a0afb9a0f1496731f63ef6f67ec5271b/Answers/Code/.ipynb_checkpoints/crypto_sentiment-checkpoint.ipynb
#https://github.com/cyonghao/NewsArticleFinder/blob/3f432d450d06487315a51830abc8bc39cdcde568/newsFinder.py
#https://github.com/2auroras/NLP-homework/blob/master/crypto_sentiment.ipynb
#https://github.com/Amefuri/pybott-3/blob/4b283b8c1bdfab40f41caa20071b2ade95245658/venv/lib/python3.8/site-packages/newsapi/newsapi_client.py#L1

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mgall\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mgall\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
#PYSPARK
# Testing local file
#data = sc.textFile("C:\\Users\\mgall\\OneDrive\\Desktop\\epa-http.txt")
#data.take(5)

In [147]:
# Initial Bitcoin Testing

# Bitcoin articles just from Google News:
#bitcoin_headlines = newsapi.get_everything(q="bitcoin", sources = 'google-news', from_param = '2020-03-21',
#                                            to = '2020-04-21',
#                                            language="en", page_size = 100, sort_by="relevancy")


#Bitcoin articles from everywhere:
bitcoin_headlines = newsapi.get_everything(q="bitcoin", from_param = '2020-03-21',
                                            to = '2020-04-21',
                                            language="en", page_size = 100, sort_by="relevancy")

# Print total articles
print(f"Total articles about Bitcoin: {bitcoin_headlines['totalResults']}")

# Show sample article
#bitcoin_headlines["articles"][0]
bitcoin_headlines["articles"]


Total articles about Bitcoin: 806


[{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
  'author': 'Romain Dillet',
  'title': 'Brave partners with Binance to let you trade crypto assets from your browser',
  'description': 'Web browser Brave is expanding its cryptocurrency features with an integration with cryptocurrency exchange Binance. Brave users will be able to buy, trade and receive crypto assets directly on the new tab page. In addition to a particular focus on privacy, B…',
  'url': 'http://techcrunch.com/2020/03/24/brave-partners-with-binance-to-let-you-trade-crypto-assets-from-your-browser/',
  'urlToImage': 'https://techcrunch.com/wp-content/uploads/2019/05/cryptocurrency.jpg?w=600',
  'publishedAt': '2020-03-24T16:00:26Z',
  'content': 'Web browser Brave is expanding its cryptocurrency features with an integration with cryptocurrency exchange Binance. Brave users will be able to buy, trade and receive crypto assets directly on the new tab page.\r\nIn addition to a particular focus on privacy, … [+1292 c

In [144]:
# Bitcoin Testing in DF
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
bitcoin_df = pd.DataFrame.from_dict(bitcoin_headlines['articles'])
bitcoin_df


,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'engadget', 'name': 'Engadget'}",Richard Lawler,WSJ: Travelex paid ransomware ring $2.3 million,Earlier this year Travelex made news after att...,https://www.engadget.com/wsj-travelex-paid-ran...,https://o.aolcdn.com/images/dims?resize=1200%2...,2020-04-09T23:39:27Z,Earlier this year Travelex made news after att...
1,"{'id': 'engadget', 'name': 'Engadget'}",Daniel Cooper,Facebook’s global currency plans were always d...,"The Libra Association, the Facebook-backed cry...",https://www.engadget.com/facebook-libra-change...,https://o.aolcdn.com/images/dims?resize=1200%2...,2020-04-16T19:00:11Z,"Now, Libra will tie itself to a series of curr..."
2,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Romain Dillet,Brave partners with Binance to let you trade c...,Web browser Brave is expanding its cryptocurre...,http://techcrunch.com/2020/03/24/brave-partner...,https://techcrunch.com/wp-content/uploads/2019...,2020-03-24T16:00:26Z,Web browser Brave is expanding its cryptocurre...
3,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Kirsten Korosec,Stocks post worst quarter since 2008 financial...,The first quarter of 2020 ended with a whimper...,http://techcrunch.com/2020/03/31/stocks-post-w...,https://techcrunch.com/wp-content/uploads/2015...,2020-03-31T20:30:58Z,The first quarter of 2020 ended with a whimper...
4,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Kirsten Korosec,Stocks blast higher on expectation of sweeping...,There are no free market fanatics on corporate...,http://techcrunch.com/2020/03/24/stocks-blast-...,https://techcrunch.com/wp-content/uploads/2019...,2020-03-24T20:21:37Z,There are no free market fanatics on corporate...
5,"{'id': None, 'name': 'Gizmodo.com'}",Catie Keck,Travelex Reportedly Paid Ransomware Hackers 28...,Following a ransomware attack against foreign ...,https://gizmodo.com/travelex-reportedly-paid-r...,https://i.kinja-img.com/gawker-media/image/upl...,2020-04-09T21:40:00Z,Following a ransomware attack against foreign ...
6,"{'id': None, 'name': 'Gizmodo.com'}",Dell Cameron,Criminal Hackers Use Official-Looking Coronavi...,Security experts are tracking a range of new t...,https://gizmodo.com/coronavirus-email-forms-us...,https://i.kinja-img.com/gawker-media/image/upl...,2020-04-01T17:51:00Z,Security experts are tracking a range of new t...
7,"{'id': None, 'name': 'Gizmodo.com'}",Janet Burns,"Will Brands Reinvent Money, or Are Digital Dol...",Many forms of so-called digital money aren’t a...,https://gizmodo.com/will-brands-reinvent-money...,https://i.kinja-img.com/gawker-media/image/upl...,2020-04-20T14:00:00Z,Many forms of so-called digital money arent ac...
8,"{'id': 'mashable', 'name': 'Mashable'}",Veronika Kero,"Get lifetime access to more than 1,000 online ...","TLD;DR: Access more than 1,000 courses with a ...",https://mashable.com/shopping/march-26-stack-s...,https://mondrian.mashable.com/2020%252F03%252F...,2020-03-26T09:00:00Z,"TLD;DR: Access more than 1,000 courses with a ..."
9,"{'id': 'mashable', 'name': 'Mashable'}",Stan Schroeder,Revolut finally launches debit card and app in...,"Revolut, one of Europe's hottest fintech start...",https://mashable.com/article/revolut-launch-us/,https://mondrian.mashable.com/2020%252F03%252F...,2020-03-24T12:13:22Z,"Revolut, one of Europe's hottest fintech start..."


In [149]:
# Initial Ethereum Testing

# Ethereum articles just from Google News:
#ethereum_headlines = newsapi.get_everything(q="ethereum", sources = 'google-news', from_param = '2020-03-21',
#                                            to = '2020-04-21',
#                                            language="en", page_size = 100, sort_by="relevancy")


#Ethereum articles from everywhere:
ethereum_headlines = newsapi.get_everything(q="ethereum", from_param = '2020-03-21',
                                            to = '2020-04-21',
                                            language="en", page_size = 100, sort_by="relevancy")

# Print total articles
print(f"Total articles about Ethereum: {ethereum_headlines['totalResults']}")

# Show sample article
#ethereum_headlines["articles"][0]
ethereum_headlines["articles"]

Total articles about Ethereum: 806


[{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
  'author': 'Romain Dillet',
  'title': 'Brave partners with Binance to let you trade crypto assets from your browser',
  'description': 'Web browser Brave is expanding its cryptocurrency features with an integration with cryptocurrency exchange Binance. Brave users will be able to buy, trade and receive crypto assets directly on the new tab page. In addition to a particular focus on privacy, B…',
  'url': 'http://techcrunch.com/2020/03/24/brave-partners-with-binance-to-let-you-trade-crypto-assets-from-your-browser/',
  'urlToImage': 'https://techcrunch.com/wp-content/uploads/2019/05/cryptocurrency.jpg?w=600',
  'publishedAt': '2020-03-24T16:00:26Z',
  'content': 'Web browser Brave is expanding its cryptocurrency features with an integration with cryptocurrency exchange Binance. Brave users will be able to buy, trade and receive crypto assets directly on the new tab page.\r\nIn addition to a particular focus on privacy, … [+1292 c

In [150]:
ethereum_df = pd.DataFrame.from_dict(ethereum_headlines['articles'])
ethereum_df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Romain Dillet,Brave partners with Binance to let you trade c...,Web browser Brave is expanding its cryptocurre...,http://techcrunch.com/2020/03/24/brave-partner...,https://techcrunch.com/wp-content/uploads/2019...,2020-03-24T16:00:26Z,Web browser Brave is expanding its cryptocurre...
1,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Danny Crichton,Securitization platform Cadence surpasses $125...,Securitization is a critical function of the m...,http://techcrunch.com/2020/04/07/securitizatio...,https://techcrunch.com/wp-content/uploads/2019...,2020-04-07T15:00:25Z,Securitization is a critical function of the m...
2,"{'id': 'the-next-web', 'name': 'The Next Web'}",Satoshi Nakaboto,Satoshi Nakaboto: ‘Daily value transfer on Eth...,Our robot colleague Satoshi Nakaboto writes ab...,https://thenextweb.com/hardfork/2020/04/16/sat...,https://img-cdn.tnwcdn.com/image/hardfork?filt...,2020-04-16T10:20:33Z,Our robot colleague Satoshi Nakaboto writes ab...
3,"{'id': None, 'name': 'Arxiv.org'}",None,The Binary Vector Clock,"The Binary Vector Clock is a simple, yet space...",https://arxiv.org/abs/2004.07087,None,2020-04-16T08:48:54Z,None
4,"{'id': 'mashable', 'name': 'Mashable'}",Stan Schroeder,Revolut finally launches debit card and app in...,"Revolut, one of Europe's hottest fintech start...",https://mashable.com/article/revolut-launch-us/,https://mondrian.mashable.com/2020%252F03%252F...,2020-03-24T12:13:22Z,"Revolut, one of Europe's hottest fintech start..."
5,"{'id': None, 'name': 'Bitcoinethereumnews.com'}",None,Bitcoin and Ethereum Source-Code Stored in a T...,Humankind just faced a huge threat from the ep...,https://bitcoinethereumnews.com/bitcoin/bitcoi...,https://cdn.coingape.com/wp-content/uploads/20...,2020-04-02T05:36:45Z,Humankind just faced a huge threat from the ep...
6,"{'id': 'business-insider', 'name': 'Business I...",Carmen Reinicke,Former hedge-fund titan Mike Novogratz says no...,Former hedge-fund titan Mike Novogratz said th...,https://markets.businessinsider.com/currencies...,https://images.markets.businessinsider.com/ima...,2020-04-13T18:41:40Z,REUTERS/Rick Wilking\r\n<ul><li>Former hedge-f...
7,"{'id': None, 'name': 'Techmeme.com'}",None,Binance launches a white paper for Binance Sma...,Yogita Khatri / The Block : Binance launches a...,https://www.techmeme.com/200417/p10,https://www.tbstat.com/wp/uploads/2019/05/Chan...,2020-04-17T16:30:01Z,About This Page\r\nThis is a Techmeme archive ...
8,"{'id': None, 'name': 'Techmeme.com'}",None,Hackers steal ~$25M worth of cryptocurrency fr...,Decrypt:\nHackers steal ~$25M worth of cryptoc...,https://www.techmeme.com/200419/p14,https://cdn.decrypt.co/wp-content/uploads/2020...,2020-04-19T20:00:39Z,None
9,"{'id': 'the-next-web', 'name': 'The Next Web'}",Satoshi Nakaboto,Satoshi Nakaboto: ‘Bitcoin price should be $25...,Our robot colleague Satoshi Nakaboto writes ab...,https://thenextweb.com/hardfork/2020/03/26/sat...,https://img-cdn.tnwcdn.com/image/hardfork?filt...,2020-03-26T10:11:40Z,Our robot colleague Satoshi Nakaboto writes ab...


In [ ]:
##############################################################################
# Experimental Testing code below (ie. Feel free to play around)

In [123]:
# More User Accessible news articles for API


# User input for country of source
country = str(input("Enter the country of choice:  "))

# Displays list of sources based on country of choice
avail_sources = newsapi.get_sources(country=country)
avail_sources = avail_sources['sources']
for avail_source in avail_sources:
    print(avail_source['id'])

# User input for selected sources
selected_sources = str(input("Enter the source(s) of your choice, each separated with a comma:   "))
selected_sources = selected_sources.strip().replace(" ","").lower()

# User input for selected keywords/ phrases
keywords = str(input("Enter keywords/ phrases to search for (use the AND / OR / NOT keywords, and optionally group these with parenthesis. Eg: crypto AND (ethereum OR litecoin) NOT bitcoin):  "))
# keywords = ",".join([keyword.strip().lower() for keyword in keywords.split(",")])
   
# User input for period of article 
# from_date = str(input("Enter the oldest date (period of article) of your choice, in YYYY-MM-DD format:  "))
# to_date = str(input("Enter the latest date (period of article) of your choice, in YYYY-MM-DD format (if none, leave blank):  "))

# Returns articles
# if to_date == "" or " ":
#     all_articles = newsapi.get_everything(q=keywords, sources=selected_sources, from_param=from_date, sort_by="relevancy")
# else:
#     all_articles = newsapi.get_everything(q=keywords, sources=selected_sources, from_param=from_date, to_param=to_date, sort_by="relevancy")

all_articles = newsapi.get_everything(q=keywords, sources=selected_sources, sort_by="relevancy")
all_articles = all_articles['articles']
for article in all_articles:
    source_name = article['source']['name']
    article_title = article['title']
    article_desc = article['description']
    article_url = article['url']
    print(source_name)
    print("Title:", article_title)
    print("Description:", article_desc)
    print("URL:", article_url)
    print("\n")

Enter the country of choice:  us
abc-news
al-jazeera-english
ars-technica
associated-press
axios
bleacher-report
bloomberg
breitbart-news
business-insider
buzzfeed
cbs-news
cnbc
cnn
cnn-es
crypto-coins-news
engadget
entertainment-weekly
espn
espn-cric-info
fortune
fox-news
fox-sports
google-news
hacker-news
ign
mashable
medical-news-today
msnbc
mtv-news
national-geographic
national-review
nbc-news
new-scientist
newsweek
new-york-magazine
next-big-future
nfl-news
nhl-news
politico
polygon
recode
reddit-r-all
reuters
techcrunch
techradar
the-american-conservative
the-hill
the-huffington-post
the-next-web
the-verge
the-wall-street-journal
the-washington-post
the-washington-times
time
usa-today
vice-news
wired
Enter the source(s) of your choice, each separated with a comma:   wired
Enter keywords/ phrases to search for (use the AND / OR / NOT keywords, and optionally group these with parenthesis. Eg: crypto AND (ethereum OR litecoin) NOT bitcoin):  cryptocurrency


In [262]:
# Experiment 1

# Trying to pull articles with specified cryptocurrency name in just title

#https://github.com/brano2/mljw/blob/e0029db3e842f8bf3599752e79d5c535085c36e0/notebooks/NewsAPI_demo.ipynb

# Sample Pulling Articles
# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q='bitcoin',
#                                           sources='bbc-news,the-verge',
                                          category='business',
                                          language='en',
                                          country='us')

# /v2/everything
all_articles = newsapi.get_everything(q='bitcoin',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param = '2020-03-21',
                                      to = '2020-04-21',
                                      language='en',
                                      sort_by='relevancy')

# /v2/sources
sources = newsapi.get_sources()

In [263]:
top_headlines['totalResults']

2

In [264]:
top_headlines['articles']

[{'source': {'id': None, 'name': 'Coindesk.com'},
  'author': 'Bradley Keoun, Omkar Godbole',
  'title': 'First Mover: 10 Takeaways for Bitcoin From Negative Oil Prices - CoinDesk - CoinDesk',
  'description': 'Oil futures crashed into negative territory, making bitcoin look like a beacon of stability. But what does the news really mean for the top cryptocurrency?',
  'url': 'https://www.coindesk.com/first-mover-10-takeaways-for-bitcoin-from-negative-oil-prices',
  'urlToImage': 'https://www.coindesk.com/wp-content/uploads/2020/04/shutterstock_1082741726-1200x628.jpg',
  'publishedAt': '2020-04-21T12:32:46Z',
  'content': 'U.S. oil futures prices turned negative Monday for the first time ever. Is it good or bad for bitcoin?\r\nThe coronavirus pandemic has so completely upended the global economy that energy demand has fallen off a cliff. People are barely driving. People are bare… [+11596 chars]'},
 {'source': {'id': None, 'name': 'Coindesk.com'},
  'author': 'Omkar Godbole',
  'title'

In [267]:
# Experiment 2

# Trying to pull articles with specified cryptocurrency name in just title

#https://github.com/brano2/mljw/blob/e0029db3e842f8bf3599752e79d5c535085c36e0/notebooks/NewsAPI_demo.ipynb

# Sample Pulling Articles
# /v2/top-headlines
top_headlines2 = newsapi.get_top_headlines(q='cryptocurrency',
                                          #sources='crypto-coins-news',
                                          category='business',
                                          language='en',
                                          country='us')

# /v2/sources
sources = newsapi.get_sources()

In [268]:
top_headlines2['totalResults']

2

In [269]:
top_headlines2['articles']

[{'source': {'id': None, 'name': 'Coindesk.com'},
  'author': 'Bradley Keoun, Omkar Godbole',
  'title': 'First Mover: 10 Takeaways for Bitcoin From Negative Oil Prices - CoinDesk - CoinDesk',
  'description': 'Oil futures crashed into negative territory, making bitcoin look like a beacon of stability. But what does the news really mean for the top cryptocurrency?',
  'url': 'https://www.coindesk.com/first-mover-10-takeaways-for-bitcoin-from-negative-oil-prices',
  'urlToImage': 'https://www.coindesk.com/wp-content/uploads/2020/04/shutterstock_1082741726-1200x628.jpg',
  'publishedAt': '2020-04-21T12:32:46Z',
  'content': 'U.S. oil futures prices turned negative Monday for the first time ever. Is it good or bad for bitcoin?\r\nThe coronavirus pandemic has so completely upended the global economy that energy demand has fallen off a cliff. People are barely driving. People are bare… [+11596 chars]'},
 {'source': {'id': None, 'name': 'Coindesk.com'},
  'author': 'Omkar Godbole',
  'title'

In [292]:
# Experiment 3

# Trying to pull articles with specified cryptocurrency name in just title

#https://github.com/Amefuri/pybott-3/blob/4b283b8c1bdfab40f41caa20071b2ade95245658/venv/lib/python3.8/site-packages/newsapi/newsapi_client.py
#https://github.com/brano2/mljw/blob/e0029db3e842f8bf3599752e79d5c535085c36e0/notebooks/NewsAPI_demo.ipynb

top_headlines3 = newsapi.get_top_headlines(q='bitcoin', language = 'en', category='business')
for article in top_headlines3['articles']:
    print('Title : ',article['title'])
    print('Description : ',article['description'],'\n\n')

Title :  En una posible estafa de salida, la billetera china de EOS retiene 52 millones de dólares de fondos de clientes - Cointelegraph (Noticias sobre el Bitcoin, Ethereum y el blockchain)
Description :  Al igual que las criptomonedas en sí mismas, las estafas relacionadas parecen estar aquí para quedarse. 


Title :  Más de USD 900,000 en Bitcoin fueron destinados a proveedores de material de abuso sexual infantil en 2019, dice Chainalysis - Cointelegraph (Noticias sobre el Bitcoin, Ethereum y el blockchain)
Description :  La empresa de inteligencia Blockchain, Chainalysis, ha publicado su último análisis sobre el uso de las criptomonedas para la compra de material de abuso sexual infantil en la darknet. 


Title :  Ripple presenta una demanda contra YouTube: "Ya es suficiente" - Cointelegraph (Noticias sobre el Bitcoin, Ethereum y el blockchain)
Description :  Ripple Labs y su director ejecutivo Brad Garlinghouse han presentado una demanda contra Youtube por su supuesta complicidad

In [288]:
top_headlines3['totalResults']

0

In [283]:
top_headlines3['articles']

[]